In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', None)
%matplotlib inline

### Nivell 1

- Exercici 1

Realitza web scraping d'una pàgina de la borsa de Madrid (https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium.

In [3]:
url = 'https://www.bolsamadrid.es'
html = requests.get(url)

In [4]:
soup = BeautifulSoup(html.content, 'html.parser')

+ anem a buscar els valors de les accions de les diferents companyies del IBEX35
+ posarem aquests valors en un dataFrame

In [5]:
# en primer lloc vaig a trobar tots els hyperlinks que tingin el string 'Acciones' 
links = []
for link in soup.find_all('a', string='Acciones'):
    links.append(link.get('href'))

In [6]:
links

['/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000',
 '/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000']

In [7]:
new_url = url+links[0]
new_url

'https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'

In [8]:
html_acciones = requests.get(new_url)
soup_acciones = BeautifulSoup(html_acciones.content, 'html.parser')

In [9]:
# busco la taula de continguts amb el seguent id especific
tabla_acciones = soup_acciones.find(id='ctl00_Contenido_tblAcciones')

In [10]:
tabla_acciones.find_all('th')

[<th scope="col">Nombre</th>,
 <th scope="col">Últ.</th>,
 <th scope="col">% Dif.</th>,
 <th scope="col">Máx.</th>,
 <th scope="col">Mín.</th>,
 <th scope="col">Volumen</th>,
 <th scope="col">Efectivo (miles €)</th>,
 <th scope="col">Fecha</th>,
 <th class="Ult" scope="col">Hora</th>]

In [11]:
columnas=[]

for element in tabla_acciones.find_all("th"):    
    columnas.append(element.get_text())
    
print(columnas)

['Nombre', 'Últ.', '% Dif.', 'Máx.', 'Mín.', 'Volumen', 'Efectivo (miles €)', 'Fecha', 'Hora']


In [12]:
# guardo tots els valors en aquestas llista
resultados = []
for element in tabla_acciones.find_all("td"):
    resultados.append(element.text)    

In [13]:
# creo un diccionari
datos = {}

In [14]:
# em salto cada 9 elements per tenir els mateix tipus de valors
for i in range(9):
    datos[columnas[i]]=resultados[i::9]

In [15]:
#converteixo el diccionari en un DataFrame
df = pd.DataFrame(datos)

In [16]:
# ja tinc la taula de les 35 companyias del IBEX
df

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"136,0000","0,22","137,8000","135,4000",59.021,"8.040,30",15/06/2021,Cierre
1,ACERINOX,"11,3500","-2,74","11,6450","11,3500",1.293.734,"14.946,43",15/06/2021,Cierre
2,ACS,"25,5400","-0,12","25,7900","25,4800",7.666.398,"197.201,71",15/06/2021,Cierre
3,AENA,"142,3500","-0,52","144,9500","141,8500",364.793,"52.306,01",15/06/2021,Cierre
4,ALMIRALL,"14,7500","-0,81","14,8600","14,6400",197.974,"2.921,75",15/06/2021,Cierre
5,AMADEUS,"64,5000","-1,53","65,9200","64,2200",589.598,"38.152,39",15/06/2021,Cierre
6,ARCELORMIT.,"25,7300","-4,47","26,9300","25,6400",939.042,"24.537,78",15/06/2021,Cierre
7,B.SANTANDER,"3,4260","-0,33","3,4430","3,3920",46.217.059,"158.172,66",15/06/2021,Cierre
8,BA.SABADELL,"0,6372","-0,50","0,6430","0,6296",18.844.863,"11.960,40",15/06/2021,Cierre
9,BANKINTER,"4,4350","-1,66","4,5300","4,4300",2.245.423,"9.982,88",15/06/2021,Cierre


In [17]:
df.to_csv('Acciones_hoy_ibex.csv')
# save data to csv

+ anem a fer una cosa similar amb selenium

In [18]:
# obrim la pagina web a la url disitjada

from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
opts = Options()
opts.set_headless()
assert opts.headless  # Operating in headless mode
#browser = Firefox(options=opts)
browser = Firefox(executable_path = '/home/vant/Descargas/geckodriver-v0.29.1-linux64/geckodriver')
url = 'https://www.bolsamadrid.es'
browser.get(url)

<ipython-input-18-a91e243008e7>:6: DeprecationWarning: use setter for headless property instead of set_headless
  opts.set_headless()


In [19]:
# busquem el link on tenim el string "Acciones" i cliquem
links = browser.find_elements_by_link_text('Acciones')
links[0].click()

In [20]:
# un cop a la nova finestra busquem els valors de la taula
table = browser.find_element_by_id('ctl00_Contenido_tblAcciones')

In [21]:
columnas=[]

for element in table.find_elements_by_tag_name("th"):    
    columnas.append(element.text)
    
print(columnas)    

['Nombre', 'Últ.', '% Dif.', 'Máx.', 'Mín.', 'Volumen', 'Efectivo (miles €)', 'Fecha', 'Hora']


In [22]:
# guardo tots els valors en aquestas llista
resultados = []
for element in table.find_elements_by_tag_name("td"):
    resultados.append(element.text)    

In [23]:
# em salto cada 9 elements per tenir els mateix tipus de valors
datos = {}
for i in range(9):
    datos[columnas[i]]=resultados[i::9]

pd.DataFrame(datos)    

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"136,0000","0,22","137,8000","135,4000",59.021,"8.040,30",15/06/2021,Cierre
1,ACERINOX,"11,3500","-2,74","11,6450","11,3500",1.293.734,"14.946,43",15/06/2021,Cierre
2,ACS,"25,5400","-0,12","25,7900","25,4800",7.666.398,"197.201,71",15/06/2021,Cierre
3,AENA,"142,3500","-0,52","144,9500","141,8500",364.793,"52.306,01",15/06/2021,Cierre
4,ALMIRALL,"14,7500","-0,81","14,8600","14,6400",197.974,"2.921,75",15/06/2021,Cierre
5,AMADEUS,"64,5000","-1,53","65,9200","64,2200",589.598,"38.152,39",15/06/2021,Cierre
6,ARCELORMIT.,"25,7300","-4,47","26,9300","25,6400",939.042,"24.537,78",15/06/2021,Cierre
7,B.SANTANDER,"3,4260","-0,33","3,4430","3,3920",46.217.059,"158.172,66",15/06/2021,Cierre
8,BA.SABADELL,"0,6372","-0,50","0,6430","0,6296",18.844.863,"11.960,40",15/06/2021,Cierre
9,BANKINTER,"4,4350","-1,66","4,5300","4,4300",2.245.423,"9.982,88",15/06/2021,Cierre


In [24]:
df.to_csv('Acciones_hoy_ibex_v2.csv')
# save data to csv

In [25]:
browser.close()

### Nivell 2

- Exercici 2

Documenta en un word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.

*** la informacio del word es pot trobar al word adjunt ****

In [27]:
df =pd.read_csv('Acciones_hoy_ibex_v2.csv')
#df.drop('Unnamed: 0',axis=1,inplace=True)

In [28]:
df.head()

,Unnamed: 0,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,0,ACCIONA,"136,0000","0,22","137,8000","135,4000",59.021,"8.040,30",15/06/2021,Cierre
1,1,ACERINOX,"11,3500","-2,74","11,6450","11,3500",1.293.734,"14.946,43",15/06/2021,Cierre
2,2,ACS,"25,5400","-0,12","25,7900","25,4800",7.666.398,"197.201,71",15/06/2021,Cierre
3,3,AENA,"142,3500","-0,52","144,9500","141,8500",364.793,"52.306,01",15/06/2021,Cierre
4,4,ALMIRALL,"14,7500","-0,81","14,8600","14,6400",197.974,"2.921,75",15/06/2021,Cierre


In [29]:
df['Últ.'] =df['Últ.'].str.replace(',','.').astype(float)
df['% Dif.'] =df['% Dif.'].str.replace(',','.').astype(float)
df['Máx.'] =df['Máx.'].str.replace(',','.').astype(float)
df['Mín.'] =df['Mín.'].str.replace(',','.').astype(float)
df['Efectivo (miles €)'] =df['Efectivo (miles €)'].str.replace('.','').str.replace(',','.').astype(float)

In [30]:
df.head()

,Unnamed: 0,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,0,ACCIONA,136.00,0.22,137.800,135.40,59.021,8040.30,15/06/2021,Cierre
1,1,ACERINOX,11.35,-2.74,11.645,11.35,1.293.734,14946.43,15/06/2021,Cierre
2,2,ACS,25.54,-0.12,25.790,25.48,7.666.398,197201.71,15/06/2021,Cierre
3,3,AENA,142.35,-0.52,144.950,141.85,364.793,52306.01,15/06/2021,Cierre
4,4,ALMIRALL,14.75,-0.81,14.860,14.64,197.974,2921.75,15/06/2021,Cierre


In [31]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          35 non-null     int64  
 1   Nombre              35 non-null     object 
 2   Últ.                35 non-null     float64
 3   % Dif.              35 non-null     float64
 4   Máx.                35 non-null     float64
 5   Mín.                35 non-null     float64
 6   Volumen             35 non-null     object 
 7   Efectivo (miles €)  35 non-null     float64
 8   Fecha               35 non-null     object 
 9   Hora                35 non-null     object 
dtypes: float64(5), int64(1), object(4)
memory usage: 2.9+ KB


### Nivell 3


- Exercici 3

Tria una página web que tu vulguis i realitza web scraping mitjançant la llibreria Scrapy. 

In [35]:
from scrapy import Selector

In [42]:
url ='https://www.elpais.es/'

In [50]:
# llegim la pagina web i guardem en contingut HTML
html = requests.get(url).content

In [51]:
# parsegem el contingut HTML amb la llibreria Selector de scrapy
sel = Selector (text = html)

In [70]:
# seleccionem tots el hyperlinks de la pagina web
links = sel.xpath ('//a/@href').extract()

In [72]:
for link in links:
    #busquem aquells links on apareixi el string "eurocopa"
    if 'eurocopa' in link:
        print(link)

https://elpais.com/deportes/eurocopa-futbol/
https://elpais.com/deportes/eurocopa-futbol/
https://resultados.elpais.com/deportivos/futbol/eurocopa/calendario/grupos_f
https://resultados.elpais.com/deportivos/futbol/eurocopa/calendario/grupos_f
/deportes/eurocopa-futbol/2021-06-11/como-de-bueno-es-cada-equipo-de-la-eurocopa-los-datos-dicen-que-espana-es-el-cuarto-mejor.html
/deportes/eurocopa-futbol/2021-06-11/como-de-bueno-es-cada-equipo-de-la-eurocopa-los-datos-dicen-que-espana-es-el-cuarto-mejor.html
https://elpais.com/deportes/eurocopa-futbol/2021-06-10/una-espana-entre-dudas.html
/deportes/eurocopa-futbol/2021-06-11/todas-las-selecciones-de-la-eurocopa.html
https://elpais.com/deportes/eurocopa-futbol/
/deportes/eurocopa-futbol/2021-06-15/francia-alemania-en-directo-la-eurocopa-en-vivo.html
https://elpais.com/deportes/eurocopa-futbol/
https://resultados.elpais.com/deportivos/futbol/eurocopa/calendario/
https://elpais.com/deportes/eurocopa-futbol/2021-06-10/una-espana-entre-dudas.htm